# Quarterly Roll Up 
[Issue](https://github.com/cal-itp/data-analyses/issues/1416)

In [1]:
import altair as alt
import _route_dir_data_prep
import _report_route_dir_visuals
import calitp_data_analysis.magics
import deploy_portfolio_yaml
import pandas as pd
import yaml
import quarterly_rollup
from IPython.display import HTML, Image, Markdown, display, display_html
from segment_speed_utils import (
    gtfs_schedule_wrangling,
    metrics,
    project_vars,
    segment_calcs,
    time_series_utils,
)
from shared_utils import (
    catalog_utils,
    gtfs_utils_v2,
    portfolio_utils,
    publish_utils,
    rt_dates,
    rt_utils,
    time_helpers,
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS


In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from omegaconf import OmegaConf

readable_dict = OmegaConf.load("readable2.yml")

In [4]:
analysis_date_list = rt_dates.y2025_dates

In [5]:
portfolio_name = "City and County of San Francisco"
# portfolio_name =  "Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"

In [6]:
ROUTE_DIR_QTR_EXPORT = 'digest/quarterly_schedule_vp_metrics_report'

In [7]:
qtr_df = pd.read_parquet(f"{RT_SCHED_GCS}{ROUTE_DIR_QTR_EXPORT}.parquet")

In [14]:
qtr_df = qtr_df.loc[(qtr_df["Portfolio Organization Name"] == portfolio_name)&(qtr_df["Route"]=="1 CALIFORNIA")]

In [15]:
qtr_df[ 'Route'].unique()

array(['1 CALIFORNIA'], dtype=object)

In [16]:
len(qtr_df)

60

In [20]:
qtr_df.columns

Index(['Direction (0/1)', 'Period', 'Average Scheduled Service (trip minutes)',
       '# Scheduled Trips', '# Realtime Trips', 'Route', 'Direction',
       '# Minutes with 1+ VP per Minute', '# Minutes with 2+ VP per Minute',
       '# Early Arrival Trips', '# On-Time Trips', '# Late Trips',
       'Average VP per Minute', '% VP within Scheduled Shape',
       '% Scheduled Trip w/ 1+ VP/Minute', '% Scheduled Trip w/ 2+ VP/Minute',
       'Realtime versus Scheduled Service Ratio', 'Speed (MPH)',
       'Portfolio Organization Name', 'Headway (Minutes)', 'GTFS Availability',
       'Average Stop Distance (Miles)', 'Quarter'],
      dtype='object')

* Avg scheduled minutes: change to quarterly.
* VP per minute: quarterly.
* Spatial Accuracy: Quarterly

In [10]:
def vp_per_minute_chart(df: pd.DataFrame) -> alt.Chart:
    specific_chart_dict = readable_dict.vp_per_min_graph
    ruler = _report_route_dir_visuals.ruler_chart(df, 3)

    bar = _report_route_dir_visuals.bar_chart(
        x_col="Quarter",
        y_col="Average VP per Minute",
        color_col="Average VP per Minute",
        color_scheme=[*specific_chart_dict.colors],
        tooltip_cols=[*specific_chart_dict.tooltip],
        date_format="",
    )

    # write this way so that the df is inherited by .facet
    chart = alt.layer(bar, ruler, data=df).properties(width=200, height=250)
    chart = chart.facet(
        column=alt.Column(
            "Direction:N",
        )
    ).properties(
        title={
            "text": specific_chart_dict.title,
            "subtitle": specific_chart_dict.subtitle,
        }
    )
    return chart


In [21]:
vp_per_minute_chart(qtr_df.loc[qtr_df["Period"]=="All Day"])

alt.FacetChart(...)

In [24]:
def avg_scheduled_min_chart(df):
    specific_chart_dict = readable_dict.avg_scheduled_min_graph
    
    chart = _report_route_dir_visuals.grouped_bar_chart(
        df,
        x_col = "Quarter",
        y_col="Average Scheduled Service (trip minutes)",
        color_col="Direction:N",
        color_scheme = [*specific_chart_dict.colors],
        tooltip_cols = [*specific_chart_dict.tooltip],
        date_format = "",
        offset_col="Direction:N",
    )
        
    chart = _report_route_dir_visuals.configure_chart(
        chart,
        width = 400, height = 250, 
        title = specific_chart_dict.title, 
        subtitle = specific_chart_dict.subtitle
    )    
    
    return chart

In [25]:
sample_avg_scheduled_min_chart(qtr_df.loc[qtr_df["Period"]=="All Day"])

alt.Chart(...)

In [26]:
def spatial_accuracy_chart(df):
    specific_chart_dict = readable_dict.spatial_accuracy_graph

    ruler = _report_route_dir_visuals.ruler_chart(df, 100)

    bar = _report_route_dir_visuals.bar_chart(
        x_col = "Quarter", 
        y_col = "% VP within Scheduled Shape", 
        color_col = "% VP within Scheduled Shape", 
        color_scheme = [*specific_chart_dict.colors], 
        tooltip_cols = [*specific_chart_dict.tooltip], 
        date_format=""
    )
   
    # write this way so that the df is inherited by .facet
    chart = alt.layer(bar, ruler, data = df).properties(width=200, height=250)
    chart = chart.facet(
        column=alt.Column(
            "Direction:N",
        )
    ).properties(
        title={
            "text": specific_chart_dict.title,
            "subtitle": specific_chart_dict.subtitle,
        }
    )
    
    return chart

In [27]:
spatial_accuracy_chart(qtr_df.loc[qtr_df["Period"]=="All Day"])

alt.FacetChart(...)